In [ ]:
import time
import os
from pathlib import Path
import ipywidgets as widgets
from enum import Enum
from pynq import Overlay, allocate
from IPython.display import display, clear_output

from axi.driver import SimpleAxiMasterDriver
from axi.enums import MemSize, MemOpStatus

# Load bitstream to overlay
BITSTREAM_PATH = '../overlay/demo.bit'

if not os.path.exists(BITSTREAM_PATH):
    raise FileNotFoundError(f'Bitstream not found at {BITSTREAM_PATH}. Run \'make bitstream\'.')

print(f'Loading overlay: {BITSTREAM_PATH}...')
ol = Overlay(BITSTREAM_PATH)

# Allocate 1024 * 4B buffer
buffer = allocate(shape=(1024,), dtype='u4')
buffer_addr = buffer.physical_address

print(f'DDR buffer allocated at {hex(buffer_addr)}')

driver = SimpleAxiMasterDriver(Path(BITSTREAM_PATH))

# Widgets
w_addr = widgets.Text(value=hex(buffer_addr), description='Addr (Hex):')
w_data = widgets.Text(value='0x12345678', description='Data (Hex):')
w_size = widgets.Dropdown(
    options=[
        ('Byte (8b)', MemSize.BYTE),
        ('Half (16b)', MemSize.HALF),
        ('Word (32b)', MemSize.WORD),
        ('Dword (64b)', MemSize.DWORD),
    ],
    value=MemSize.WORD,
    description="Size:"
)
w_log = widgets.Output(layout={
    'border': '1px solid black',
    'height': '150px',
    'overflow_y': 'scroll',
})

btn_write = widgets.Button(description='WRITE', button_style='warning')
btn_read = widgets.Button(description='READ', button_style='success')
btn_check = widgets.Button(description='Check', button_style='info')

# Callbacks
def on_write_click(_):
    with w_log:
        try:
            addr = int(w_addr.value, 16)
            data = int(w_data.value, 16)
            size = w_size.value

            print(f'> WRITE: Addr={hex(addr)} Data={hex(data)} Size={size}')

            status = driver.write(size, addr, data)

            if status != MemOpStatus.DONE:
                print('  RESULT: ERROR')
            else:
                print('  RESULT: DONE')

        except Exception as e:
            print(f'  ERROR: {e}')

def on_read_click(_):
    with w_log:
        try:
            addr = int(w_addr.value, 16)
            size = w_size.value

            print(f'> READ: Addr={hex(addr)} Size={size}')

            buffer.invalidate()

            result = driver.read(size, addr)

            if result.status != MemOpStatus.DONE:
                print('  RESULT: ERROR')
            else:
                print(f'  RESULT: Data = {hex(result.data)}')

        except Exception as e:
            print(f'  ERROR: {e}')

def on_check(b):
    with w_log:
        print(f'[Debug] Buffer[0] = {hex(buffer[0])}')
        print(f'[Debug] Buffer[1] = {hex(buffer[1])}')

btn_write.on_click(on_write_click)
btn_read.on_click(on_read_click)
btn_check.on_click(on_check)

# Display
ui = widgets.VBox(
    [
        widgets.HTML('<h3>AXI Master Demo</h3>'),
        w_addr,
        w_data,
        w_size,
        widgets.HBox(
            [btn_write, btn_read, btn_check]
        ),
        w_log
    ]
)

driver.clear()
display(ui)
